# New Section

## Load libraries

In [ ]:
import numpy as np
import sympy as sp

## Assumptions and Variables

1. We assign the base a diameter of say 60m
2. The external legs are evenly spaced some distance away from the edge
3. From our view, the angle each leg makes with the horizontal, can be calculated as follows. Given length of leg $l_i$ for $ i \in {1, 2, 3}$, length of central leg $l_c$, and radius of base $r_b$.
4. The legs at the extreme make two angles with the horizontal, α and β (along $XZ$, $YZ$ respectively), α = β. α can be calculated as $α = sin^{-1}(l_c/l_i)$
5. The leg aligned with the central leg makes an angle $β = sin^{-1}(l_c/l_i)$ along $YZ$.



In [ ]:
# Configurations
ra = 0.4  # radius of base (aka static platform)
rb = 0.1  # radius of mobile platform (aka dynamic platform)

d = 10E-2   # diameter of link
# for 3 legs
L1 = [0.8]*4
L2 = [0.8]*4
# for central leg
Lc1 = [0.8]*4
Lc2 = [0.8]*4

############################################
S = np.pi*(d**2)/4
Iy = np.pi*(d**2)/64
Iz = np.pi*(d**2)/64
J = Iy + Iz
############################################

# beam cross section area 
# external legs
Se1 = S
Se2 = S
# central legs
Sc1 = S
Sc2 = S

# Second Order Moment
# External Legs
Iy1 = 0.03  # x principle moment of inertia for bottom beam
Iy2 = 0.04  # y principle moment of inertia for top beam
Iz1 =  0.04
Iz2 = 0.05
# Central Legs
Icy1 = 0.05  # x principle moment of inertia for bottom beam
Icy2 = 0.04  # y principle moment of inertia for top beam
Icz1 =  0.06
Icz2 = 0.02

# Polar Moments
# External
J1 = 0.5 # top leg
J2 = 0.6
# Central
Jc1 = 0.5
J2c = 0.55

# Young and Coulomb's Modulus
# External
Ee1 = 70E9 # lower
Ee2 = 70E9 # upper
Ge1 = 25E9  # lower
Ge2 = 25E9    # upper
# Central
Ec = 70E9
Gc = 25E9



# Main Code

## Rotation Matrices

We have a rotation matrix for each of the 3 legs.\
$Leg_1$ is rotated along the $x$ and $y$ axis. \
$Leg_2$ is also rotated along the $x$ and $y$ axis. \
$Leg_3$ is rotated along the $x$ axis only. \
$Leg_c$ (the central leg) is not rotated. \


In [ ]:
q1, q2, q3 = sp.symbols('q1, q2, q3')

Rx = sp.Matrix([[1, 0, 0],
               [0, sp.cos(q1), -sp.sin(q1)] ,
               [0, sp.sin(q1), sp.cos(q1)]])
Ry = sp.Matrix([[sp.cos(q1), 0, sp.sin(q1)],
               [0, 1, 0],
               [-sp.sin(q1), 0, sp.cos(q1)]])
Rz = sp.Matrix([[sp.cos(q1), -sp.sin(q1), 0], 
               [sp.sin(q1), sp.cos(q1), 0],
               [0, 0, 1]])

# a, b = sp.symbols('α, β')
# Rx.subs(q1, a)@Ry.subs(q1, b)

## Kinematics

In [ ]:
def get_lengths(a, b, h, H):
    c = np.cos
    s = np.sin
    r3 = np.sqrt(3)
    l1 = ( ((rb*c(b/2) + h*s(b) - 0.5*ra)**2) + 
            (rb*s(a)*s(b/2) - r3*rb*c(a/2) - h*s(a)*c(b) + r3*ra/2)**2 +
            (-rb*c(a)*s(b/2) - r3*ra*s(a/2) + h*c(a)*c(b) + H)**2)

    l2 = (
            ((rb*c(b/2) + h*s(b) - 0.5*ra)**2) + 
            (rb*s(a)*s(b/2) + r3*rb*c(a/2) - h*s(a)*c(b) - r3*ra/2)**2 + 
            (-rb*c(a)*s(b/2) + r3*ra*s(a/2) + h*c(a)*c(b) + H)**2
          )
    
    l3 = (
            (-rb*c(b) + h*s(b) + ra)**2 + 
            (-rb*s(a)*s(b) - h*s(a)*c(b))**2 + 
            (-rb*c(a)*s(b) + h*c(a)*c(b) + H)**2
            )
    return l1+0.6, l2+0.6, l3+0.6

In [ ]:
# h has a ax of 0.8 so we set it to a value between  0.6 and 0.8
# alpha and beta values values between +-np.pi/6

# for i in range
l1s = []
l2s = []
l3s = []

lc = 0.6
l1, l2, l3 = get_lengths(np.pi/6, np.pi/8, lc, 0)

In [ ]:
def get_rotation_angle(l1=0, l2=0, lc1=0, lc2=0):
    """
    A function that takes in link lengths l1, l2, lc1, lc2, 
    And calculates the angle l1+l2 makes with the horizontal.
    """
    lc = lc1+lc2
    li = l1+l2
    theta = np.arcsin(lc/li) # * (180/np.pi) # second part is to convert to degrees
    return theta

def get_rotation_matrix(theta: float, axis:list):
    """
    A function that thats in an angle q, 
    And returns a 3x3 rotation matrix 
    Gotten by taking the rotation of the angle around X, Y or/and Z.
    """
    d = {'x': Rx, 'y': Ry, 'z': Rz}
    axis.reverse()
    expr = d[axis.pop()]
    while len(axis) != 0:
        expr = expr@d[axis.pop()]
    val = expr.subs(q1, theta)
    return val

# ang = get_rotation_angle(1, 3, 1, 2)
# d = get_rotation_matrix(ang, ['x']).evalf()
# d

def rotate(R):
    M = np.zeros((6, 6))
    M[:3, :3] = R
    M[3:, 3:] = R
    return M

## Stiffness Matrix

Next we get our $K$ matrix

In [ ]:
E, S, L, G, J, Iy, Iz = sp.symbols('E, S, L, G, J, Iy, Iz')

_K22 = sp.Matrix([
               [E*S/L, 0, 0, 0, 0, 0],
               [0, 12*E*Iz/(L**3), 0, 0, 0, -6*E*Iz/(L**2)],
               [0, 0, 12*E*Iy/(L**3), 0, 6*E*Iy/(L**2), 0],
               [0, 0, 0, G*J/L, 0, 0],
               [0, 0, 6*E*Iy/(L**2), 0, 4*E*Iy/L, 0],
               [0, -6*E*Iz/(L**2), 0, 0, 0, 4*E*Iz/L],
])

_K11 = sp.Matrix([
               [E*S/L, 0, 0, 0, 0, 0],
               [0, 12*E*Iz/(L**3), 0, 0, 0, 6*E*Iz/(L**2)],
               [0, 0, 12*E*Iy/(L**3), 0, -6*E*Iy/(L**2), 0],
               [0, 0, 0, G*J/L, 0, 0],
               [0, 0, -6*E*Iy/(L**2), 0, 4*E*Iy/L, 0],
               [0, 6*E*Iz/(L**2), 0, 0, 0, 4*E*Iz/L],
])

_K12 = sp.Matrix([
               [-E*S/L, 0, 0, 0, 0, 0],
               [0, -12*E*Iz/(L**3), 0, 0, 0, 6*E*Iz/(L**2)],
               [0, 0, -12*E*Iy/(L**3), 0, -6*E*Iy/(L**2), 0],
               [0, 0, 0, -G*J/L, 0, 0],
               [0, 0, 6*E*Iy/(L**2), 0, 2*E*Iy/L, 0],
               [0, -6*E*Iz/(L**2), 0, 0, 0, 2*E*Iz/L],
])

In [ ]:
def get_stiffness(values: tuple):
    """
    Takes in E, S, L, G, J, Iy, Iz
    returns K11, K12, K22
    K21 = K12'
    """
    k11_lambda = sp.lambdify([E, S, L, G, J, Iy, Iz], _K11, 'numpy')
    k12_lambda = sp.lambdify([E, S, L, G, J, Iy, Iz], _K12, 'numpy')
    k22_lambda = sp.lambdify([E, S, L, G, J, Iy, Iz], _K22, 'numpy')

    return k11_lambda(*values), k12_lambda(*values), k22_lambda(*values)


## Lambdas

In [ ]:
# Lambda rigid
# Translational
I = np.eye(6)
l_rx_p = np.delete(I, 0, 0) # delete row 4
l_ry_p = np.delete(I, 1, 0)
l_rz_p = np.delete(I, 2, 0)
# Rotational
l_rx_R = np.delete(I, 3, 0) # delete row 4
l_ry_R = np.delete(I, 4, 0)
l_rz_R = np.delete(I, 5, 0)
# Spherical
l_p_S = np.delete(I, slice(0,3), 0)
l_r_S = np.delete(I, slice(3,6), 0)

# Lambda passive and elastic
# Translational
l_px_P = np.zeros((1,6))
l_px_P[0][0] = 1
l_py_P = np.zeros((1,6))
l_py_P[0][1] = 1
l_pz_P = np.zeros((1,6))
l_pz_P[0][2] = 1
# Rotational
l_px_R = np.zeros((1,6))
l_px_R[0][3] = 1
l_py_R = np.zeros((1,6))
l_py_R[0][4] = 1
l_pz_R = np.zeros((1,6))
l_pz_R[0][5] = 1

l_ex_P = l_px_P
l_ex_R = l_px_R
l_ey_P = l_py_P
l_ey_R = l_py_R
l_ez_P = l_pz_P
l_ez_R = l_pz_R

l_rxy_R = np.delete(I, slice(3,5), 0)
l_pxy_R = np.zeros((2,6))
l_pxy_R[:,3:5] = np.eye(2)

## Aggregated Matrix and $K_c$ for External Chain

In [ ]:
def calculate_external_stiffness(lu, lb, R):
    l = lu # can index them
    # Upper Legs
    K11, K12, K22 = get_stiffness((Ee1, Se1, l, Ge1, J1, Iy1, Iz1))
    M = rotate(R)
    K11 = M@K11@M.T
    K21 = K12.T

    K_links = np.zeros((24, 24))

    K_links[0:6,0:6] = K11
    K_links[0:6,6:12] = K12
    K_links[6:12,0:6] = K21
    K_links[6:12,6:12] = K22
    # Lower Legs
    l = lb # can index them
    K11, K12, K22 = get_stiffness((Ee2, Se2, l, Ge1, J2, Iy2, Iz2))
    K11 = M@K11@M.T
    K21 = K12.T

    K_links[0:6,12:18] = K11
    K_links[0:6,18:24] = K12
    K_links[6:12,12:18] = K21
    K_links[6:12,18:24] = K22

    # Aggregated Passive Joints
    agg_links = np.hstack([np.zeros((24, 6)), -np.eye(24), np.zeros((24, 6)), K_links])

    # Aggregated Passive Joints
    agg_passive_joints = np.zeros((12, 60))
    agg_passive_joints[0:3,30:36] = l_r_S
    agg_passive_joints[0:3,36:42] = -l_r_S

    agg_passive_joints[3:6,0:6] = l_r_S
    agg_passive_joints[3:6,6:12] = l_r_S

    agg_passive_joints[6:9,0:6] = l_p_S
    agg_passive_joints[9:12,6:12] = l_p_S
    agg_links.shape

    # Aggregated Active Joints
    # Given stiffness of joint K_a
    K_a = 1000000
    agg_actuated_joints = np.zeros((12, 60))
    agg_actuated_joints[0:5,42:48] = l_rz_p
    agg_actuated_joints[0:5,42:48] = -l_rz_p
    #
    agg_actuated_joints[5:11,12:18] = I
    agg_actuated_joints[5:11,18:24] = I
    #
    agg_actuated_joints[11:12,12:18] = l_ez_P
    # K_a, we take only the row where there is rotation, z in this case
    agg_actuated_joints[11:12,12:18] = K_a*l_ez_P
    agg_actuated_joints[11:12,42:48] = -K_a*l_ez_P

    # Aggregated Rigid Base
    agg_base = np.zeros((6, 60))
    agg_base[:,30:36] = I

    # External Loading
    agg_ext = np.zeros((6, 60))
    agg_ext[:,24:30] = I

    final_agg = np.vstack([agg_links, agg_passive_joints, agg_actuated_joints, agg_base, agg_ext])

    A = final_agg[:54, :54]
    B = final_agg[:54, 54:]
    C = final_agg[54:, :54]
    D = final_agg[54:, 54:]

    Kci = D - C@np.linalg.pinv(A)@B
    return Kci

In [ ]:
lc = 0.6
l1, l2, l3= get_lengths(np.pi/6, np.pi/8, lc, 0)

a = get_rotation_angle(l1=l1, lc1=lc)
R = get_rotation_matrix(a, ['x', 'y'])
sp.Matrix(R)
# print(a, l1, lc, lc/l1)
# print
Kc1 = calculate_external_stiffness(l1/2, l1/2, R)
a = get_rotation_angle(l1=l2, lc1=lc)
R = get_rotation_matrix(a, ['x'])
Kc2 = calculate_external_stiffness(l2/2, l2/2, R)
a = get_rotation_angle(l1=l3, lc1=lc)
R = get_rotation_matrix(a, ['x', 'y'])
Kc3 = calculate_external_stiffness(l3/2, l3/2, R)

## Aggregated Matrix and $K_c$ for Central Leg

In [ ]:
l = Lc1[0] # can index them
# Upper Legs
K11, K12, K22 = get_stiffness((Ec, Sc1, l, Gc, Jc1, Icy1, Icz1))
K21 = K12.T

K_links = np.zeros((24, 24))

K_links[0:6,0:6] = K11
K_links[0:6,6:12] = K12
K_links[6:12,0:6] = K21
K_links[6:12,6:12] = K22

l = Lc2[0] # can index them
# Upper Legs
K11, K12, K22 = get_stiffness((Ec, Sc1, l, Gc, Jc1, Icy1, Icz1))
K21 = K12.T

K_links[0:6,12:18] = K11
K_links[0:6,18:24] = K12
K_links[6:12,12:18] = K21
K_links[6:12,18:24] = K22

# Aggregated Passive Joints
agg_links = np.hstack([np.zeros((24, 6)), -np.eye(24), np.zeros((24, 6)), K_links])

# Aggregated Rigid Joints
agg_rigid_joints = np.zeros((12, 60))
agg_rigid_joints[0:6,30:36] = I
agg_rigid_joints[0:6,36:42] = -I

agg_rigid_joints[6: ,0:6] = I
agg_rigid_joints[6: ,6:12] = I

agg_links.shape

# Aggregated Active Joints
# Given stiffness of joint K_a
K_a = 1000000
agg_actuated_joints = np.zeros((12, 60))
agg_actuated_joints[0:5,42:48] = l_rz_p
agg_actuated_joints[0:5,42:48] = -l_rz_p
#
agg_actuated_joints[5:11,12:18] = I
agg_actuated_joints[5:11,18:24] = I
#
agg_actuated_joints[11:12,12:18] = l_ez_P
# K_a, taking only the row where there is rotation, z in this case
agg_actuated_joints[11:12,12:18] = K_a*l_ez_P
agg_actuated_joints[11:12,42:48] = -K_a*l_ez_P

# Aggregated Rigid Base
agg_base = np.zeros((6, 60))
agg_base[:,30:36] = I

# External Loading
agg_ext = np.zeros((6, 60))
agg_ext[:,24:30] = I

final_agg = np.vstack([agg_links, agg_rigid_joints, agg_actuated_joints, agg_base, agg_ext])

A = final_agg[:54, :54]
B = final_agg[:54, 54:]
C = final_agg[54:, :54]
D = final_agg[54:, 54:]

Kcc = D - C@np.linalg.pinv(A)@B

## Mobile Platform and Manipulator Stiffness

In [ ]:
def skew(x):
    a = np.zeros((3, 3))
    a[0][1] = -x[2]
    a[0][2] =  x[1]
    a[1][2] = -x[0]

    a[1][0] =  x[2]
    a[2][0] = -x[1]
    a[2][1] =  x[0]
    return a

def get_block_matrix(d):
    D = np.eye(6)
    D[3:, :3] = np.zeros((3,3))
    D[:3, 3:] = d.T
    return D

In [ ]:
# Given radius of platform
r = rb
sqr = (r**2+r**2) - (2*r*r*np.cos(120))
c = np.sqrt(sqr)
x = c/2
h = c**2 - x**2
y = h - r
D1 = get_block_matrix(skew([x, y, 0]))
D2 = D1
D3 = get_block_matrix(skew([0, r, 0]))
Dc = get_block_matrix(np.zeros((3, 3)))

In [ ]:
# Platform
agg_p = np.zeros((30, 60))
#
agg_p[:6, 30:36] = -I
agg_p[:6, 54:] = D1
#
agg_p[6:12, 36:42] = -I
agg_p[6:12, 54:] = D2
#
agg_p[12:18, 42:48] = -I
agg_p[12:18, 54:] = D3
#
agg_p[18:24, 48:54] = -I
agg_p[18:24, 54:] = Dc
################## new arrangement##########
agg_p[24:30, :6] = D1.T
agg_p[24:30, 6:12] = D2.T
agg_p[24:30, 12:18] = D3.T
agg_p[24:30, 18:24] = Dc.T
agg_p[24:30, 24:30] = I

# Joints
agg_j = np.zeros((26, 60))
agg_j[:2, :6] = l_pxy_R
agg_j[2:4, 6:12] = l_pxy_R
agg_j[4:6, 12:18] = l_pxy_R
agg_j[6:8, 18:24] = l_pxy_R
#
agg_j[8:12, :6] = -l_rxy_R
agg_j[8:12, 30:36] = l_rxy_R@Kc1 # 1
agg_j[12:16, 6:12] = -l_rxy_R
agg_j[12:16, 36:42] = l_rxy_R@Kc2 # 2
agg_j[16:20, 12:18] = -l_rxy_R
agg_j[16:20, 42:48] = l_rxy_R@Kc3 # 3
agg_j[20:24, 18:24] = -l_rxy_R
agg_j[20:24, 48:54] = l_rxy_R@Kcc # central

agg = np.vstack([agg_j, agg_p])

D = agg[50:,54:]
C = agg[50:,:54]

A = agg[:50,:54]
B = agg[:50,54:]

# print(A.shape, B.shape, C.shape, D.shape)

Kc = D - C@np.linalg.pinv(A)@B
# print(np.linalg.matrix_rank(Kc))
sp.Matrix(Kc)

Matrix([
[ 1.25961413809776e-8, -2.14497422561961e-5,  2.23882032625721e-5, -9.71868740676861e-6,  3.34299192399017e-6,  2.60896260426523e-6],
[-4.28306311856035e-8,  9.23202804438162e-7,  4.01573439913559e-6, -4.24702130161175e-6,  1.58577440276711e-6, -1.37433135221706e-6],
[ 2.50117672793353e-8,  1.18936873176082e-5,  1.56892553061422e-5,  7.96383103497999e-6,  4.78737206306364e-6, -5.73095070208872e-7],
[ 4.21387689426397e-9, -1.96660589562193e-6, -8.71607197310267e-7,  5.44554883238524e-7, -1.02596216549551e-7,  2.73466944208943e-7],
[3.67284395851579e-10, -5.10740776479745e-7,  -2.5699442880567e-7, -5.06498010757486e-8, -4.32727710290389e-8,  6.67279720800721e-8],
[ 1.21164469085484e-7, -6.53756902736765e-5,  1.58497605209353e-5,  1.81512383317742e-5,   6.0297683936447e-6,  9.43840915269246e-6]])

## Deflections

To calculate the deflection at the end-effector, we use Hooke's Law. 
\begin{equation}
W = K_cΔt
\end{equation}
From which deflections can be gotten as
\begin{equation}
Δt = K_c^{-1}W
\end{equation}

In [ ]:
# We apply a force of 1000N along the Z axis and calculate the deflection
W = np.array([[0, 1000, 0, 0, 0, 0]]).T
dt = np.linalg.inv(Kc)@W
sp.Matrix(dt)

Matrix([
[ 6635076185.47547],
[-80800821.3641135],
[-61501877.2332501],
[ -28363886.482242],
[ 331184582.112723],
[ -698600948.99881]])

# Appendix

Code Stubs

In [ ]:
for i in ('x', 'y', 'z'):
    print(f'l_e{i}_P = l_p{i}_P')
    print(f'l_e{i}_R = l_p{i}_R')

l_ex_P = l_px_P
l_ex_R = l_px_R
l_ey_P = l_py_P
l_ey_R = l_py_R
l_ez_P = l_pz_P
l_ez_R = l_pz_R


In [ ]:
e = np.zeros((6, 6))
e[1:3,0:3] = 9
e = np.arange(36).reshape((6, 6))
print(e)
print(e[:5, :5])
print(e[:5, 5:])

[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]
 [24 25 26 27 28 29]
 [30 31 32 33 34 35]]
[[ 0  1  2  3  4]
 [ 6  7  8  9 10]
 [12 13 14 15 16]
 [18 19 20 21 22]
 [24 25 26 27 28]]
[[ 5]
 [11]
 [17]
 [23]
 [29]]


# References




1.  [Delete From an Array](https://note.nkmk.me/en/python-numpy-delete/)
2.   [Lamdify](https://docs.sympy.org/latest/modules/utilities/lambdify.html)
3. [Height of Isosceles Triangle](https://study.com/skill/learn/how-to-inscribe-an-equilateral-triangle-in-a-circle-explanation.html)


